In [1]:
## VICTOR DICE:
import os
import gc
import time
import requests
import subprocess
import random as rn
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import socks
import socket
import pymssql
import numpy as np
import pandas as pd

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

### SOLO ES NECESARIO CORRER UNA VEZ

In [2]:
#!pip install socks

In [3]:
#!pip install pymssql

### OPCIONES

In [4]:
alternativo = 0
priorizado = 1

In [5]:
print(dir(socks))

['BytesIO', 'Callable', 'DEFAULT_PORTS', 'EAGAIN', 'EINVAL', 'EOPNOTSUPP', 'GeneralProxyError', 'HTTP', 'HTTPError', 'PRINTABLE_PROXY_TYPES', 'PROXY_TYPES', 'PROXY_TYPE_HTTP', 'PROXY_TYPE_SOCKS4', 'PROXY_TYPE_SOCKS5', 'ProxyConnectionError', 'ProxyError', 'SEEK_CUR', 'SOCKS4', 'SOCKS4Error', 'SOCKS4_ERRORS', 'SOCKS5', 'SOCKS5AuthError', 'SOCKS5Error', 'SOCKS5_ERRORS', '_BaseSocket', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_makemethod', '_orgsocket', '_orig_socket', 'b64encode', 'create_connection', 'functools', 'get_default_proxy', 'getdefaultproxy', 'log', 'logging', 'method', 'name', 'os', 'set_default_proxy', 'set_self_blocking', 'setdefaultproxy', 'socket', 'socksocket', 'struct', 'sys', 'wrap_module', 'wrapmodule']


In [6]:
print(socks.getdefaultproxy())

None


In [7]:
now = datetime.now()

desfase_2_meses =  datetime.strptime((now - relativedelta(months=2)).strftime('%Y-%m'), '%Y-%m').strftime('%Y%m')
desfase_2_meses

'202101'

In [8]:
letras = "abcdefghijklmnopqrstuvwyz"
user_aletorio = ''.join([rn.choice(letras) for _ in range(8)])
pass_aleatorio = ''.join([rn.choice(letras) for _ in range(8)])

user_aletorio, pass_aleatorio

('tokjtfbj', 'gmwcpnou')

In [9]:
users_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36 Edg/88.0.705.74",
    "Mozilla/5.0 CK={} (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
]

url = "http://api.sunat.binvoice.net/consulta.php"

headers = {
     'content-type': "application/x-www-form-urlencoded",
     'cache-control': "no-cache",
     'User-Agent': rn.choice(users_agents),
     'Referer': "http://api.sunat.binvoice.net/",
     'Origin': "api.sunat.binvoice.net",
     'Upgrade-Insecure-Requests': '0'
}

In [1]:
#import asyncdns, asyncio

ModuleNotFoundError: No module named 'asyncdns'

In [10]:
try:
    connection_market.close()
except:
    pass


#conneciom MARKET
connection_market = pymssql.connect(host='10.11.12.90\BDT',
                             user='ibetlmarket',
                             port=1433,
                             password='m@rk3t2o15',
                             database='MARKET'
                            )
time.sleep(1)
gc.collect()

263

### QUERY COMPLETO > 16 000 CLIENTES

In [11]:
sql_all_rucs_clientes = """
    SELECT DISTINCT(NUM_RUC) FROM ODS.HM_VPC_CLIENTE
    WHERE 1 = 1
    and PERIODO >= '202007'
    and NUM_RUC LIKE '20%'
    and (
        ZONAL in ('BEL ZONAL 1','BEL ZONAL 2','BEL ZONAL 3','BEP ZONAL 1',
                  'BEP ZONAL 2','BEP ZONAL 3', 'BC GRUPO 2', 'BC GRUPO 1')
        OR
        BANCA in ('BE','BEL','BEP', 'BC')
    )
"""

### QUERY PRIORIZADO ALERTAS CARTERA  < 3000

In [12]:
sql_priorizado = """
    SELECT DISTINCT(NUM_RUC)FROM ODS.HD_VMVP_BASE_ALERTA_CARTERA_BE
    WHERE 1 = 1
    and FUNNEL_ALERTA = 1
    and ZONAL IS NOT NULL
    and ZONAL != ''
    and PERIODO >= '202007'
"""

In [13]:
rucs = pd.read_sql(
   sql_priorizado if priorizado else sql_all_rucs_clientes, connection_market
)
print(rucs.shape)

connection_market.close()
rucs.head()

(2811, 1)


,NUM_RUC
0,20548187266
1,20505688903
2,20523552903
3,20114022838
4,20393679809


In [14]:
rucs = list(rucs['NUM_RUC'].unique())
len(rucs)

2811

### IDENTIFICACION quienes no registran trabajadores

In [15]:
csv_not = 'not_workers.csv'
not_workers = pd.read_csv(csv_not, dtype={'RUC': str}, parse_dates=['FECHA']).sort_values(
    by=['FECHA'], ascending=False).drop_duplicates(subset=['RUC'], keep='first')

### Vuelve a revisar los detectados hace 2 meses
not_workers = not_workers[
    not_workers['FECHA'] > now - relativedelta(months=2)
]

not_workers.to_csv(csv_not, index=False)
not_workers = pd.read_csv(csv_not, dtype={'RUC': str}, parse_dates=['FECHA'])
not_workers.head()

,RUC,FECHA
0,20518400542,2021-02-27 09:46:57.752043
1,20507225382,2021-02-27 09:46:55.335756
2,20606112867,2021-02-27 09:46:52.875315
3,10180741696,2021-02-27 09:46:50.111419
4,20545048613,2021-02-27 09:46:47.650677


In [16]:
not_workers.dtypes

RUC              object
FECHA    datetime64[ns]
dtype: object

In [17]:
list_not = list(not_workers['RUC'].unique())
len(list_not)

61

### AUTO limpiado de duplicados

In [18]:
csv_name = "cantidad_trabajadores.csv"
cant_trab = pd.read_csv(csv_name, dtype={'RUC': str, 'PERIODO': str})
print(cant_trab.shape)

cant_trab = cant_trab.drop_duplicates(subset=['PERIODO', 'RUC'], keep='last')
cant_trab.to_csv(csv_name, index=False)
print(cant_trab.shape)

(98684, 9)
(98684, 9)


### DETETCCIÓN de clientes a buscar actualización

In [51]:
cant_trab = pd.read_csv(csv_name, dtype={'RUC': str, 'PERIODO': str, 'ESTADO': str})
print(cant_trab.shape)
cant_trab.head()

(98684, 9)


,PERIODO,ANIO,MES,TOTAL_TRABAJADORES,PENSIONISTA,PRESTADOR_SERVICIO,RUC,COMBINACION,ESTADO
0,202001,2020,1,172,0,24,20530611681,202001__20530611681,NaN
1,202002,2020,2,174,0,22,20530611681,202002__20530611681,NaN
2,202003,2020,3,154,0,25,20530611681,202003__20530611681,NaN
3,202004,2020,4,149,0,15,20530611681,202004__20530611681,NaN
4,202005,2020,5,146,0,19,20530611681,202005__20530611681,NaN


In [52]:
cant_trab.dtypes

PERIODO                object
ANIO                    int64
MES                     int64
TOTAL_TRABAJADORES     object
PENSIONISTA            object
PRESTADOR_SERVICIO     object
RUC                    object
COMBINACION            object
 ESTADO               float64
dtype: object

In [53]:
cant_trab['PERIODO'].value_counts(dropna=False)

202004    7731
202008    7731
202002    7731
202005    7731
202009    7731
202011    7731
202007    7731
202003    7731
202012    7731
202006    7731
202001    7731
202010    7731
201912    2056
201911    1939
201910    1916
202101       1
Name: PERIODO, dtype: int64

In [54]:
clientes_nuevos = set(rucs) - set(cant_trab['RUC'])
len(clientes_nuevos)

55

In [55]:
clientes_a_actualizar = set(cant_trab['RUC']) - set(cant_trab[cant_trab['PERIODO'] == desfase_2_meses]['RUC'])
len(clientes_a_actualizar)

7730

In [56]:
sep = '__'
clientes_update = list(
    (clientes_nuevos | clientes_a_actualizar) - set(list_not)
)
del clientes_nuevos
del clientes_a_actualizar

len(clientes_update)

7733

In [57]:
rucs_periodo_existentes = list(cant_trab['COMBINACION'].unique())
del cant_trab

#len(rucs_periodo_existentes)

In [58]:
gc.collect()

66

In [59]:
rn.shuffle(clientes_update)

if alternativo:
    print("OPEN")
    sproc = subprocess.Popen("C:\\Users\\Villacorta\\Desktop\\Tor Browser\\Browser\\firefox.exe")

In [60]:
agents_tor = [
    'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
]

headers_tor = {
    'content-type': "application/x-www-form-urlencoded",
    'User-Agent': rn.choice(agents_tor),
}

def get_response(ruc):
    user_aletorio = ''.join([rn.choice(letras) for _ in range(8)])
    pass_aleatorio = ''.join([rn.choice(letras) for _ in range(8)])
    
    socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9150, True, user_aletorio, pass_aleatorio)
    socket.socket = socks.socksocket
    
    headers_tor.update(
        {
            'User-Agent': rn.choice(agents_tor),
            'X-Forwarded-For': requests.get('http://ifconfig.me/ip').text
        }
    )
    #print()
    
    print(headers_tor)
    response = requests.request("POST", url, data="nruc={}".format(ruc), headers=headers_tor)
    #print(response)
    return response

In [61]:
def get_response_simple(ruc):
    headers.update(
        {'User-Agent': rn.choice(users_agents)}
    )
    response = requests.request("POST", url, data="nruc={}".format(ruc), headers=headers)
    #print(response)
    return response

In [44]:
for ruc in clientes_update:
    response = get_response(ruc) if alternativo else  get_response_simple(ruc)
    try:
        data = response.json()
    except:
        print(" {} ".format(response.status_code))
        time.sleep(60)
        gc.collect()
        continue
    
    if not data.get("success"):
        print(" er ", data.get("message", ""), end=' ')
        time.sleep(60)
        gc.collect()
        continue
    
    estado = data.get('result',{}).get('estado', '')
    print(estado)
    cantidad_trabajadores = data.get('result',{}).get('cantidad_trabajadores',{})
    cantidad_trabajadores = pd.DataFrame(cantidad_trabajadores) if len(cantidad_trabajadores) > 0 else None
    
    ### SI NO TIENE TRABAJADORES REGISTRAODS, lo guardamos aparte para no buscarlo en otras corridas
    if not isinstance(cantidad_trabajadores, pd.core.frame.DataFrame):
        print(ruc, 
              " nt ", end='')
        
        pd.DataFrame(
            [[ruc, datetime.now()]]
        ).to_csv(
            csv_not, 
            mode='a',
            header=False, 
            index=False
        )
        
        gc.collect()
        continue
    
    ### FORMATO
    cantidad_trabajadores.columns = [col.upper() for col in cantidad_trabajadores.columns]
    cantidad_trabajadores['PERIODO'] = cantidad_trabajadores['PERIODO'].apply(lambda _: _.replace('-', '').replace('_', ''))
    cantidad_trabajadores['RUC'] = ruc
    cantidad_trabajadores['COMBINACION'] = cantidad_trabajadores['PERIODO'] + sep +  cantidad_trabajadores['RUC']
    cantidad_trabajadores['ESTADO'] = estado
    ### GUARDADO, y acumulación de los satisfactorios
    cantidad_trabajadores[
        ~cantidad_trabajadores['COMBINACION'].isin(rucs_periodo_existentes)
    ].to_csv(
        csv_name, 
        mode='a',
        header=False, 
        index=False
    )
    print(ruc, end=' ')


if alternativo:
    sproc.kill()

ACTIVO
20600946090  nt ACTIVO
20307150981  nt ACTIVO
20116447321  nt ACTIVO
20265681299  nt ACTIVO
20450688542  nt ACTIVO
20601625769  nt ACTIVO
20101266819  nt ACTIVO
20513139846  nt 

KeyboardInterrupt: 